In [28]:
from __future__ import annotations

import numpy as np
import matplotlib.pyplot as plt

import random
import time

import attr
from attr.validators import instance_of

from pathlib import Path
import os

from typing import List, Dict, NewType, Callable, Any, Iterable, SupportsIndex

In [46]:
T_BenchName = NewType("bench_name", str)
T_dt = NewType("dt", float)
T_dts = NewType("dts", List[T_dt])

@attr.s(frozen=True)
class BenchmarkDesc:
    name: T_BenchName = attr.ib()
    n_execs: int = attr.ib()
    total_time: float = attr.ib()
    mean: float = attr.ib()
    median: float = attr.ib()
    std: float = attr.ib()


@attr.s(frozen=True)
class BenchmarkData:
    """
    Attributes:
    -----------
    - `name (str):` Nombre del benchmark.
    - `dts (List[dt]):` Guarda el `dt` de cada ejecución.
    - `n_execs (int):`  len(dts) ~~ Número de ejecuciones.
    - `is_empty (bool):` n_execs==0 -> True.
    - `total_time (float):` sum(dts) ~~ Suma de todos los tiempos de ejecución.
    - `mean (float):`   np.mean(dts) ~~ Media.
    - `median (float):` np.median(dts) ~~ Mediana.
    - `std (float):` np.std(dts) ~~ Desv. Standard.
    - `description (BenchmarkDesc)`: Retorna un diccionario con los valores representativos.

    Methods:
    --------
    """
    name: T_BenchName = attr.ib(validator=instance_of(str))
    dts: T_dts = attr.ib(validator=instance_of(list), default=[])

    def __len__(self) -> int:
        return len(self.dts)
    
    @property
    def n_execs(self) -> int:
        return len(self)
    
    @property
    def is_empty(self) -> bool:
        return self.n_execs == 0

    @property
    def total_time(self) -> float:
        return self.__none_if_empty(np.sum(self.dts))
    
    @property
    def mean(self) -> float:
        return self.__none_if_empty(np.mean(self.dts))
    
    @property
    def median(self) -> float:
        return self.__none_if_empty(np.median(self.dts))
    
    @property
    def std(self) -> float:
        return self.__none_if_empty(np.std(self.dts))

    @property
    def description(self) -> BenchmarkDesc:
        return BenchmarkDesc(
            name = self.name,
            n_execs = self.n_execs,
            total_time = self.total_time,
            mean = self.mean,
            median = self.median,
            std = self.std
        )

    def append(self, dt: T_dt) -> None:
        self.dts.append(dt)

    def reset(self) -> None:
        """ Elimina todos los `dts` guardados."""
        self.dts.clear()
    
    def __none_if_empty(self, value: Any) -> Any | None:
        return value if not self.is_empty else None

'''
class Benchmarks(Dict[T_BenchName, BenchmarkData]):
    """ Benchmark de todas las funciones decoradas con `dt`."""
    def __init__(self, bench_names: Iterable[str]):
        assert len(bench_names) == len(set(bench_names)), "No puede haber claves repetidas."
        super().__init__({name: [] for name in bench_names})

    @property
    def time_now(self) -> float:
        return time.perf_counter()

    def dt(self, name: T_BenchName) -> Callable[..., Any]:
        """
        Decorador: Calcula el tiempo que tarda en ejecutar la función decorada.
        - name (str): Key donde se almacena 
        - Se almacena dentro de `Benchmark` con key `name` en una lista.
        """
        def decorator(func: Callable[..., Any]) -> Callable[..., Any]:
            def wrapper(*args, **kwargs):
                t_i = self.time_now
                result = func(*args, **kwargs)
                dt = self.time_now - t_i
                self.add_dt(name, dt)
                return result
            return wrapper
        return decorator

    def add_dt(self, bench_name: T_BenchName, dt: T_dt) -> None:
        """ Appendea `dt` dentro de `bench_name`."""
        self[bench_name].append(dt)
    
    def description(self, bench_name: T_BenchName) -> BenchmarkData:
        """ Retorna la descripción de un benchmark en concreto."""
        return BenchmarkData.from_dts(bench_name, self[bench_name])

    def descriptions(self) -> List[BenchmarkData]:
        """ Retorna la descripción de todos los benchmarks."""
        return [self.description(bench_name) for bench_name in self.keys()]

    def reset(self, bench_name: T_BenchName = None):
        """ """
        pass

    def plot(self, bench_name: str) -> None:
        """ TODO: Mejorar, no cuentro la forma rápida de hacer esto automático."""
        plt.hist(self[bench_name], log=True)
        plt.show()'''
'''
benchs = Benchmarks(("hola", "funcion_costosa"))



@benchs.dt(name="hola")
def sum_n_numbers(n: int):
    sum([random.randint(0,10) for _ in range(n)])

for _ in range(10000):
    sum_n_numbers(1000)

asd = benchs.descriptions()'''


'\nbenchs = Benchmarks(("hola", "funcion_costosa"))\n\n\n\n@benchs.dt(name="hola")\ndef sum_n_numbers(n: int):\n    sum([random.randint(0,10) for _ in range(n)])\n\nfor _ in range(10000):\n    sum_n_numbers(1000)\n\nasd = benchs.descriptions()'

In [47]:
b = BenchmarkData("asdasd", [3,4])
b.description

BenchmarkDesc(name='asdasd', n_execs=2, total_time=7, mean=3.5, median=3.5, std=0.5)